In [1]:
!python -m pip install xgboost

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from tqdm import tqdm
from utils import LoadRumoursDataset,LoadRumoursDatasetFilterNodeonTestV2
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pylab as plt
import uuid
from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import *
from imblearn.over_sampling import SMOTE

In [1]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.set_experiment("spyder-experiment")
import mlflow.pytorch
#mlflow.set_experiment("Xgboost experiments 2024-09-26 SMOTE")

In [3]:
#
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"


### SMOTE APPROACH

In [11]:
dataset_generator  = LoadRumoursDatasetFilterNodeonTestV2(file_path_replies, file_path_posts,time_cut_posts=60*3*24,\
                                                          time_cut_replies=60*24*2)
dataset_generator.load_data()
dataset_generator.process_data()
train_dataset, test_dataset = dataset_generator.get_final_dataframes()

In [12]:
train_dataset

,followers,favorite_count,retweet_count,first_time_diff,replies,no_verified,verified,embeddings_avg,rumour
0,-0.085680,1.458015,0.528139,0.240550,0.333333,1,0,"[0.20999414314116752, 0.13815921917557716, 0.0...",0
1,0.362618,7.877863,4.316017,-0.292096,0.666667,0,1,"[0.14064250700175762, 0.0028862979263067244, -...",0
2,0.232104,0.366412,6.705628,-0.426117,1.000000,0,1,"[0.07760866896973716, 0.05833733040425512, 0.2...",1
3,-0.076887,-0.045802,0.043290,0.487973,0.333333,1,0,"[0.020305360401315347, 0.037071573947157176, 0...",0
4,23.473225,0.717557,0.714286,-0.371134,0.666667,0,1,"[-0.1835014725724856, -0.2997356637613848, 0.0...",1
...,...,...,...,...,...,...,...,...,...
1396,-0.084104,-0.488550,-0.350649,2.443299,-0.777778,1,0,"[-0.31757273025472055, -0.17581527815623718, 0...",1
1397,0.722883,9.198473,10.406926,-0.371134,0.333333,0,1,"[-0.031781600415706636, 0.0024340026080608367,...",0
1398,0.010306,1.511450,0.831169,-0.024055,1.111111,1,0,"[0.11681999691895076, 0.2433300060885293, 0.37...",0
1399,0.177065,0.198473,-0.233766,-0.271478,0.333333,0,1,"[0.18432844802737236, 0.23420833320253426, 0.3...",0


In [13]:
X_train  = train_dataset.drop(columns=['rumour'])
X_train = np.hstack([X_train.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X_train.embeddings_avg.tolist()))])


X_test  = test_dataset.drop(columns=['rumour'])
X_test = np.hstack([X_test.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X_test.embeddings_avg.tolist()))])

#X = np.hstack([X.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X.embeddings_avg.tolist()))])
y_train =train_dataset['rumour']
y_test =test_dataset['rumour']

In [14]:
print(y_train.sum(),y_train.shape)

313 (1401,)


In [15]:
print(y_test.sum(),y_test.shape)

134 (601,)


In [16]:
# Apply SMOTE to the features without embeddings
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42,sampling_strategy='minority')
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [17]:
X_resampled.shape

(2176, 107)

In [18]:
print(y_resampled.sum(),y_resampled.shape)

1088 (2176,)


#### Hyperparameter tunning

In [19]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats

# Define the hyperparameter distributions
param_dist = {
     'max_depth': [3, 4, 5, 6],
    'learning_rate': np.linspace(0.001, 0.1, 20),
    'n_estimators': [20,50, 100, 150, 200]
}

# Create the XGBoost model object
xgb_model = xgb.XGBClassifier()

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=20, cv=5, scoring='f1')


In [20]:
# Fit the RandomizedSearchCV object to the training data
random_search.fit(X_resampled, y_resampled)

# Print the best set of hyperparameters and the corresponding score
print("Best set of hyperparameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

Best set of hyperparameters:  {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.07394736842105264}
Best score:  0.9398873449167626


In [21]:
best_params = random_search.best_params_

In [22]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score

def modelfit(alg, X_train, y_train, X_test, y_test, performCV=True, printFeatureImportance=True, cv_folds=5):
    # Fit the algorithm on the data
    #alg.fit(X_train, y_train)
        
    # Predict training set:
    dtrain_predictions = alg.predict(X_train)
    dtrain_predprob = alg.predict_proba(X_train)[:,1]

    # Predict test set:
    dtest_predictions = alg.predict(X_test)
    dtest_predprob = alg.predict_proba(X_test)[:,1]
    
    # Perform cross-validation:
    if performCV:
        cv_score = cross_val_score(alg, X_train, y_train, cv=cv_folds, scoring='recall')
    
    # Print model report for Train:
    print("\nModel Report Train")
    print("AUC Score : %f" % roc_auc_score(y_train, dtrain_predprob))
    print("Accuracy : %.4g" % accuracy_score(y_train, dtrain_predictions))
    print("Precision : %.4g" % precision_score(y_train, dtrain_predictions))
    print("Recall : %f" % recall_score(y_train, dtrain_predictions))
    

    
    # Print model report for Test:
    print("\nModel Report Test")
    print("AUC Score : %f" % roc_auc_score(y_test, dtest_predprob))
    print("Accuracy : %.4g" % accuracy_score(y_test, dtest_predictions))
    print("Precision : %.4g" % precision_score(y_test, dtest_predictions))
    print("Recall : %f" % recall_score(y_test, dtest_predictions))
    

    
    plt.tight_layout()
    plt.show()
    
    if performCV:
        print("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score), np.std(cv_score), np.min(cv_score), np.max(cv_score)))

    return alg



In [29]:
best_params={
             'n_estimators': 50, 
             'max_depth': 3, 
             'learning_rate': 0.05, 
       }

In [23]:
# Train the final model with the best hyperparameters
final_model = xgb.XGBClassifier(
    eval_metric='logloss',
    **best_params
)
final_model.fit(X_resampled, y_resampled)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.07394736842105264, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [24]:
modelfit(final_model,X_resampled, y_resampled,X_test,y_test)


Model Report Train
AUC Score : 1.000000
Accuracy : 1
Precision : 1
Recall : 1.000000

Model Report Test
AUC Score : 0.915577
Accuracy : 0.8502
Precision : 0.6375
Recall : 0.761194


<Figure size 640x480 with 0 Axes>

CV Score : Mean - 0.9788737 | Std - 0.02683625 | Min - 0.9266055 | Max - 1


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.07394736842105264, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [25]:
modelfit(final_model,X_train, y_train,X_test,y_test)


Model Report Train
AUC Score : 1.000000
Accuracy : 1
Precision : 1
Recall : 1.000000

Model Report Test
AUC Score : 0.915577
Accuracy : 0.8502
Precision : 0.6375
Recall : 0.761194


<Figure size 640x480 with 0 Axes>

CV Score : Mean - 0.6226318 | Std - 0.06425552 | Min - 0.516129 | Max - 0.7142857


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.07394736842105264, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

#### Without SMOTE

In [26]:
# Train the final model with the best hyperparameters
final_model = xgb.XGBClassifier(
    eval_metric='logloss',
    **best_params
)
final_model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.07394736842105264, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [27]:
modelfit(final_model, X_train, y_train,X_test,y_test)


Model Report Train
AUC Score : 1.000000
Accuracy : 1
Precision : 1
Recall : 1.000000

Model Report Test
AUC Score : 0.915993
Accuracy : 0.8586
Precision : 0.696
Recall : 0.649254


<Figure size 640x480 with 0 Axes>

CV Score : Mean - 0.6226318 | Std - 0.06425552 | Min - 0.516129 | Max - 0.7142857


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.07394736842105264, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

#### SMOTE and generic Xgboost pipeline

In [9]:
# Train the final model with the best hyperparameters
params =   {
    'objective': 'binary:logistic',
    'max_depth': 5,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'min_child_weight': 1,
    'lambda': 1,
    'alpha': 0.5
}
final_model = xgb.XGBClassifier(
    eval_metric='logloss',
    **params
)
final_model.fit(X_resampled, y_resampled)


XGBClassifier(alpha=0.5, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.1, grow_policy=None,
              importance_type=None, interaction_constraints=None, lambda=1,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, ...)

In [29]:
modelfit(final_model, X_train, y_train,X_test,y_test)


Model Report Train
AUC Score : 0.998958
Accuracy : 0.9793
Precision : 0.9176
Recall : 0.996805

Model Report Test
AUC Score : 0.906133
Accuracy : 0.8386
Precision : 0.6082
Recall : 0.776119


<Figure size 640x480 with 0 Axes>

CV Score : Mean - 0.6195597 | Std - 0.06782318 | Min - 0.5322581 | Max - 0.7142857


XGBClassifier(alpha=0.5, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.1, grow_policy=None,
              importance_type=None, interaction_constraints=None, lambda=1,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, ...)

#### General Parameters:

* objective: "binary:logistic" (since it's a binary classification problem)
*  eval_metric: "logloss" (logarithmic loss is a good measure for classification)
  
#### Tree Booster Parameters:
* max_depth: 4 (shallower trees to prevent overfitting)
* learning_rate: 0.05 (a lower learning rate helps the model generalize better)
* n_estimators: 100 (start with fewer trees to prevent overfitting on a small dataset)
* subsample: 0.8 (use 80% of the data for each boosting round to introduce randomness)
* colsample_bytree: 0.8 (use 80% of the features, useful with high-dimensional embeddings)

#### Regularization Parameters:
* gamma: 0.1 (penalty for additional leaf nodes, controls tree complexity)
* min_child_weight: 1 (controls the minimum sum of instance weight in a child, useful for preventing overfitting)
* lambda: 1 (L2 regularization to reduce overfitting)
* alpha: 0.5 (L1 regularization for feature selection)
  


In [21]:
def modelfit(alg, X_train, y_train, X_test, y_test, printFeatureImportance=True):
    with mlflow.start_run():
        # Fit the algorithm on the data
        #alg.fit(X_resampled, y_resampled)
            
        # Predict training set:
        dtrain_predictions = alg.predict(X_train)
        dtrain_predprob = alg.predict_proba(X_train)[:, 1]

        # Predict test set:
        dtest_predictions = alg.predict(X_test)
        dtest_predprob = alg.predict_proba(X_test)[:, 1]

        # Log model parameters
        mlflow.log_params(alg.get_params())

        # Log metrics for Train
        mlflow.log_metric("train_auc", roc_auc_score(y_train, dtrain_predprob))
        mlflow.log_metric("train_accuracy", accuracy_score(y_train, dtrain_predictions))
        mlflow.log_metric("train_precision", precision_score(y_train, dtrain_predictions))
        mlflow.log_metric("train_recall", recall_score(y_train, dtrain_predictions))

        # Log metrics for Test
        mlflow.log_metric("test_auc", roc_auc_score(y_test, dtest_predprob))
        mlflow.log_metric("test_accuracy", accuracy_score(y_test, dtest_predictions))
        mlflow.log_metric("test_precision", precision_score(y_test, dtest_predictions))
        mlflow.log_metric("test_recall", recall_score(y_test, dtest_predictions))

        # Print model report for Train
        print("\nModel Report Train")
        print("AUC Score : %f" % roc_auc_score(y_train, dtrain_predprob))
        print("Accuracy : %.4g" % accuracy_score(y_train, dtrain_predictions))
        print("Precision : %.4g" % precision_score(y_train, dtrain_predictions))
        print("Recall : %f" % recall_score(y_train, dtrain_predictions))

      

        # Log confusion matrix for Train
        #train_cm = confusion_matrix(y_train, dtrain_predictions)
        #mlflow.log_artifact(plot_confusion_matrix(train_cm, "Train"))

        # Print model report for Test
        print("\nModel Report Test")
        print("AUC Score : %f" % roc_auc_score(y_test, dtest_predprob))
        print("Accuracy : %.4g" % accuracy_score(y_test, dtest_predictions))
        print("Precision : %.4g" % precision_score(y_test, dtest_predictions))
        print("Recall : %f" % recall_score(y_test, dtest_predictions))


        # Log the model with input example
        #input_example = X_train[0:1]   # Use the first row of X_train as an example
        #mlflow.sklearn.log_model(alg, "model", input_example=input_example)
        mlflow.log_metric("time_cut_replies", time_cut_replies)
        mlflow.log_metric("time_cut_posts", time_cut_posts)
    
    # Explicitly end the MLflow run
    mlflow.end_run()

    return alg



In [19]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.set_experiment("spyder-experiment")
import mlflow.pytorch
mlflow.set_experiment("Xgboost SMOTE Filter Node on test two time cuts 2024-10-17 20 min-3days")

<Experiment: artifact_location='/home/azureuser/rumour-detection-pheme/mlruns/42', creation_time=1729211340368, experiment_id='42', last_update_time=1729211340368, lifecycle_stage='active', name='Xgboost SMOTE Filter Node on test two time cuts 2024-10-17 20 min-3days', tags={}>

In [ ]:
#
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"

params =   {
    'objective': 'binary:logistic',
    'max_depth': 5,
    'learning_rate': 0.05,
    'n_estimators': 100,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'min_child_weight': 1,
    'lambda': 1,
    'alpha': 0.5
}

for time_cut_posts in range(2058,(60*24*3),60):
    if time_cut_posts == 2058:
        start_replies = 1177+30
    else:
        start_replies = 7
    for time_cut_replies in range(start_replies,(60*24*2),30):
        print(time_cut_posts,time_cut_replies)
        
        dataset_generator  = LoadRumoursDatasetFilterNodeonTestV2(file_path_replies, file_path_posts,time_cut_posts,\
                                                          time_cut_replies)
        dataset_generator.load_data()
        dataset_generator.process_data()
        train_dataset, test_dataset = dataset_generator.get_final_dataframes()
        
        X_train  = train_dataset.drop(columns=['rumour'])
        X_train = np.hstack([X_train.drop(columns=['embeddings_avg']).values, \
                             np.array(pd.DataFrame(X_train.embeddings_avg.tolist()))])


        X_test  = test_dataset.drop(columns=['rumour'])
        X_test = np.hstack([X_test.drop(columns=['embeddings_avg']).values, \
                            np.array(pd.DataFrame(X_test.embeddings_avg.tolist()))])

        #X = np.hstack([X.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X.embeddings_avg.tolist()))])
        y_train =train_dataset['rumour']
        y_test =test_dataset['rumour']
        
        #if time_cut_posts >20:

        smote = SMOTE(random_state=42,sampling_strategy='minority')
        X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

        final_model = xgb.XGBClassifier(
        eval_metric='logloss',
        **params
        )
        
        final_model.fit(X_resampled, y_resampled)
         
        
        modelfit(final_model,  X_train, y_train,X_test,y_test)
        
        

2058 1207

Model Report Train
AUC Score : 0.998958
Accuracy : 0.9793
Precision : 0.9176
Recall : 0.996805

Model Report Test
AUC Score : 0.931932
Accuracy : 0.8889
Precision : 0.6735
Recall : 0.750000
2058 1237

Model Report Train
AUC Score : 0.998958
Accuracy : 0.9793
Precision : 0.9176
Recall : 0.996805

Model Report Test
AUC Score : 0.931932
Accuracy : 0.8889
Precision : 0.6735
Recall : 0.750000
2058 1267

Model Report Train
AUC Score : 0.998958
Accuracy : 0.9793
Precision : 0.9176
Recall : 0.996805

Model Report Test
AUC Score : 0.933993
Accuracy : 0.8907
Precision : 0.68
Recall : 0.755556
2058 1297

Model Report Train
AUC Score : 0.998958
Accuracy : 0.9793
Precision : 0.9176
Recall : 0.996805

Model Report Test
AUC Score : 0.936084
Accuracy : 0.8898
Precision : 0.6923
Recall : 0.750000
2058 1327

Model Report Train
AUC Score : 0.998958
Accuracy : 0.9793
Precision : 0.9176
Recall : 0.996805

Model Report Test
AUC Score : 0.934902
Accuracy : 0.8868
Precision : 0.6792
Recall : 0.7346

In [16]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
# Get the experiment ID or name
experiment_name = "Xgboost SMOTE Filter Node on test two time cuts 2024-10-17 20 min-3days"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is not None:
    experiment_id = experiment.experiment_id

    # Retrieve all runs in the experiment as a DataFrame
    df = mlflow.search_runs(experiment_ids=[experiment_id])
    
    # Show the DataFrame
    #print(df)
else:
    print(f"Experiment '{experiment_name}' not found.")


In [10]:
df

,run_id,experiment_id,status,artifact_uri,start_time,end_time


In [17]:
df['metrics.time_cut_posts'].max()

2058.0

In [18]:
df[df['metrics.time_cut_posts']==2058]['metrics.time_cut_replies'].max()

1177.0